# AG3- Actividad Guiada 3<br>
(he incluido también Colonia de Hormigas y GA)

Nombre: Íñigo Ballester Gúrpide<br>
Colab: https://colab.research.google.com/drive/10rWiCQ69FO9tlK0-tQDm0O6yap9_SZbd?usp=sharing<br>
Github: https://github.com/ibagur/03MIAR-algoritmos-de-optimizacion-2021/tree/main/AG3


In [ ]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

## Preparación

In [ ]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file) 

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/eil51.tsp", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/att48.tsp", file)




('swiss42.tsp', <http.client.HTTPMessage at 0x7f5ff0305f90>)

In [ ]:
#Modulos extras, no esenciales
import numpy as np
import matplotlib.pyplot as plt
import imageio                    #Para construir las imagenes con gif
from google.colab import files    #Para descargar ficheros generados con google colab

from tempfile import mkstemp      #Para genera carpetas y ficheros temporales

import random                     #Para generar valores aleatorios

from math import e

import copy     #Permite hacer copias de objetos en python: listas, diccionarios,...

In [ ]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())



In [ ]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
#dir(problem)

15

In [ ]:
 
#Funcionas basicas
###############################################################################      
     
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion 

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)


 len(crear_solucion(Nodos))

42

## Búsqueda Aleatoria

In [ ]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto
  
  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)
    
    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia
      
      
  print("Mejor solución:" , mejor_solucion) 
  print("Distancia     :" , mejor_distancia) 
  return mejor_solucion  

    
#Busqueda aleatoria con 5000 iteraciones    
solucion = busqueda_aleatoria(problem, 5000)    
 

Mejor solución: [0, 34, 13, 20, 12, 3, 1, 15, 17, 5, 29, 39, 8, 26, 18, 4, 10, 21, 40, 38, 28, 22, 9, 6, 14, 30, 37, 2, 27, 11, 23, 24, 32, 31, 25, 41, 7, 19, 16, 33, 35, 36]
Distancia     : 3848


## Búsqueda Local

In [ ]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################

def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):
      
      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]  

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)         

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))
 

nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

 

Distancia Solucion Incial: 3848
Distancia Mejor Solucion Local: 3543


In [ ]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []
  
  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)
 
    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia 
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion) 
      print("Distancia     :" , mejor_distancia) 
      return mejor_solucion
      
    solucion_referencia = vecina
 
 
sol = busqueda_local(problem )  


En la iteracion  28 , la mejor solución encontrada es: [0, 19, 13, 11, 25, 10, 8, 41, 23, 9, 29, 4, 6, 5, 26, 18, 27, 32, 34, 20, 31, 17, 7, 1, 28, 30, 22, 39, 21, 12, 14, 16, 15, 37, 3, 2, 40, 24, 38, 33, 35, 36]
Distancia     : 2139


## Recocido Simulado

In [ ]:
###############################################################################
# SIMULATED ANNEALING
###############################################################################

#Generador de 1 solucion vecina 2-opt 100% aleatoria (intercambiar 2 nodos)
#Mejorable eligiendo otra forma de elegir una vecina.
def genera_vecina_aleatorio(solucion):

  #Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))
  
  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  
 
genera_vecina_aleatorio(solucion) 


[0,
 34,
 13,
 20,
 12,
 3,
 1,
 15,
 17,
 5,
 29,
 26,
 8,
 39,
 18,
 4,
 10,
 21,
 40,
 38,
 28,
 22,
 9,
 6,
 14,
 30,
 37,
 2,
 27,
 11,
 23,
 24,
 32,
 31,
 25,
 41,
 7,
 19,
 16,
 33,
 35,
 36]

In [ ]:

#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99
 



In [ ]:
def recocido_simulado(problem, TEMPERATURA ):
  #problem = datos del problema
  #T = Temperatura
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  
  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    vecina =genera_vecina_aleatorio(solucion_referencia)
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
      
    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    #Si la nueva vecina es mejor se cambia  
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      #solucion_referencia = copy.deepcopy(vecina)
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)
 
  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol  = recocido_simulado(problem, 10000000)

 



La mejor solución encontrada es [0, 7, 37, 36, 2, 25, 41, 29, 28, 32, 20, 34, 8, 10, 5, 14, 16, 15, 1, 6, 3, 4, 12, 11, 18, 17, 31, 35, 33, 38, 21, 24, 40, 23, 9, 39, 22, 30, 27, 26, 13, 19]
con una distancia total de 2291


## Algoritmo de Colonia de Hormigas

In [ ]:
#DATOS DEL PROBLEMA
file = "swiss42.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file) 
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion(lista de nodos)
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

La función Add_Nodo selecciona al azar un nodo con probabilidad uniforme.
Para ser mas eficiente debería seleccionar el próximo nodo siguiendo la probabilidad correspondiente a la ecuación:

$p^k_{ij}(t) = \frac{[\tau_{ij}(t)]^\alpha[\nu_{ij}]^\beta}{\sum_{l\in J^k_i} [\tau_{il}(t)]^\alpha[\nu_{il}]^\beta}$, si $j \in J^k_i$

$p^k_{ij}(t) = 0$, si $j \notin J^k_i$

In [ ]:
def Add_Nodo(problem, H ,T ) :
  #Mejora:Establecer una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )


def Incrementa_Feromona(problem, T, H ) :
  #Incrementa segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T ):
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  #Mejora:Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [ ]:
def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
  #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  #Mejora: inicializar con valores diferentes dependiendo diferentes criterios
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
  
  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)
  
  
hormigas(problem, 1000)

[0, 29, 26, 18, 19, 30, 8, 23, 41, 2, 9, 31, 4, 1, 14, 27, 39, 24, 21, 10, 13, 25, 28, 11, 12, 36, 17, 34, 40, 32, 33, 6, 37, 16, 5, 38, 22, 7, 20, 35, 15, 3]
3774


## Algoritmo Genético para el TSP

In [ ]:
#Librerias y carga del problema

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # https://wwwproxy.iwr.uni-heidelberg.de/groups/comopt/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/usage.html
  # https://tsplib95.readthedocs.io/modules.html#module-tsplib95.models

#Matriz de distancias
file = "swiss42.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

#Objeto de tsplib95 para nuestro problema problema
problem = tsplib95.load_problem(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

#Coordenadas(si estan disponibles en el ficher)
problem.get_display(1)

#Distancia 
problem.get_weight(1, 2)

#Matriz de distancias
[ [problem.get_weight(i, j)   for i in  list(problem.get_nodes()) ]  for j in  list(problem.get_nodes()) ]  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated function (or staticmethod) load_problem. (Will be removed in newer versions. Use `tsplib95.load` instead.) -- Deprecated since version 7.0.0.
  del sys.path[0]


[[0,
  15,
  30,
  23,
  32,
  55,
  33,
  37,
  92,
  114,
  92,
  110,
  96,
  90,
  74,
  76,
  82,
  67,
  72,
  78,
  82,
  159,
  122,
  131,
  206,
  112,
  57,
  28,
  43,
  70,
  65,
  66,
  37,
  103,
  84,
  125,
  129,
  72,
  126,
  141,
  183,
  124],
 [15,
  0,
  34,
  23,
  27,
  40,
  19,
  32,
  93,
  117,
  88,
  100,
  87,
  75,
  63,
  67,
  71,
  69,
  62,
  63,
  96,
  164,
  132,
  131,
  212,
  106,
  44,
  33,
  51,
  77,
  75,
  72,
  52,
  118,
  99,
  132,
  132,
  67,
  139,
  148,
  186,
  122],
 [30,
  34,
  0,
  11,
  18,
  57,
  36,
  65,
  62,
  84,
  64,
  89,
  76,
  93,
  95,
  100,
  104,
  98,
  57,
  88,
  99,
  130,
  100,
  101,
  179,
  86,
  51,
  4,
  18,
  43,
  45,
  95,
  45,
  115,
  93,
  152,
  159,
  100,
  112,
  114,
  153,
  94],
 [23,
  23,
  11,
  0,
  11,
  48,
  26,
  54,
  70,
  94,
  69,
  89,
  75,
  84,
  84,
  89,
  92,
  89,
  54,
  78,
  99,
  141,
  111,
  109,
  190,
  89,
  44,
  11,
  29,
  54,
  56,
  89,
  47,
  1

In [ ]:
#Funciones de la Actividad Guiada 3
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [Nodos[0]]
  for n in Nodos[1:] :
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion 

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)


#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [ ]:
#Genera una poblacion inicial de soluciones de tamaño N
def generar_poblacion(Nodos,N):
  return [crear_solucion(Nodos) for _ in range(N) ]


#Evalua la población y devuelve el mejor individuo
def Evaluar_Poblacion(poblacion, problem):
  mejor_solucion = []
  mejor_distancia = 10e100
  for p in poblacion:
    #print("solucion:", p)
    distancia_referencia = distancia_total(p, problem)
    if distancia_referencia < mejor_distancia:
      mejor_solucion = p
      mejor_distancia = distancia_referencia
  return mejor_solucion, mejor_distancia   

#Funcion de cruce. Recibe una poblacion(lista de soluciones) y devuelve la población ampliada con los hijos.
# Todos los individuos de la población son selecionados para el cruce(si la población es par)
# Podría aplicarse un proceso previo de selección para elegir los individuos que se desea cruzar.
def Cruzar(poblacion,mutacion, problem):
  #Definimos en una variable la copia de la población para ir eliminando los padres seleccionados
  poblacion_copia = copy.deepcopy(poblacion)

  #Definimos en una variable la copia de la población para ir añadiendo los hijos creados
  poblacion_final = copy.deepcopy(poblacion)

  while len(poblacion_copia) > 1:  #Iteramos mientras haya padres disponibles
    #Seleccionamos dos padres
    padre1,padre2 = random.sample(poblacion_copia   ,2)
    poblacion_copia.remove(padre1)
    poblacion_copia.remove(padre2)
    poblacion_final.extend(Descendencia([padre1,padre2],problem,mutacion))
  return poblacion_final

#Funcion para generar hijos a partir de 2 padres:
# Se elige el metodo de 1-punto de corte pero es posible usar otros n-puntos, uniforme, dependiendo del problema
def Descendencia(padres, problem,mutacion):
  #Se elige un punto de corte aleatorio:
  pc = random.sample( range(1,len(padres[0])),1)[0]
  hijo1 =  Factibilizar(padres[0][:pc] + padres[1][pc:] ,problem) 
  hijo2 =  Factibilizar(padres[1][:pc] + padres[0][pc:] ,problem)
  return [hijo1,hijo2,Mutar(hijo1,mutacion),Mutar(hijo2,mutacion)]

#Para el operador de cruce 1-punto los hijos generados no son soluciones(algunos nodos se repiten y otros no están)
def Factibilizar(solucion,problem):
  Nodos = list(problem.get_nodes())
  nodos_desaparecidos = list(set(Nodos) - set(solucion))
  #Recorremos todos los nodos, cuando haya uno que ya esté en la lista los cambiamos por uno de la lista de nodos_desaparecidos
  for i in range(len(solucion) ):
    if solucion[i] in solucion[:i]:
      #print("\tSe repite el",solucion[i] )
      #print("\tSe cambia en la posicion ",i , " " ,solucion[i] , " por ", nodos_desaparecidos[0] )
      solucion[i] = nodos_desaparecidos.pop(0) #Cambiamos el nodo y a la vez eliminamos el nodo usado de nodos_desaparecidos
  return solucion 

#Funcion de mutación. Se eligen dos nodos y se intercambia. Se podrian añadir otros operaradores
# Se hace mutaciones mutacion% de las veces
def Mutar(solucion,mutacion):
  if random.random() < mutacion:
    sel1,sel2 = sorted(random.sample(set(solucion)-{solucion[0]},2))
    return solucion[:sel1] + [solucion[sel2]] + solucion[sel1+1:sel2] + [solucion[sel1]]  + solucion[sel2+1:] 
  else:
    return solucion[::] 

#Funcion de seleccion de la población. Recibe como parametro una poblacion y 
# devuelve una poblacion a la que se ha eliminado individuos poco aptos(fitness alto) y para mantener una poblacion estable de N individuos
#Se tiene en cuenta el porcentaje elitismo pasado como parametro 
# Para los individuos que no son de la elite podríamos usar una selección de ruleta(proporcional a su fitness)
def Seleccionar(problem,poblacion, N, elitismo):
  #Se ordena la población según el fitness(tamaño del recorrido) en una lista de elementos [distancia, solucion]
  poblacion_ordenada = sorted([ [distancia_total(solucion, problem), solucion] for solucion in poblacion ], key= lambda x:x[0] )

  #Devolvemos elitismo% y el resto se eligen aleatoriamente
  return [x[1] for x in poblacion_ordenada][:int(N*elitismo)]  +   random.sample([x[1] for x in poblacion_ordenada][int(N*elitismo):] , int(N*(1-elitismo))  ) 
  

In [ ]:
#Funcion principal del algoritmo genetico
#######################################################3
def algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=100):
  # problem = datos del problema
  # N = Tamaño de la población
  # mutacion = probabilidad de una mutación
  # elitismo = porcion de la mejor poblacion a mantener
  # generaciones = nº de generaciones a generar para finalizar

  #Genera la poblacion inicial
  Nodos = list(problem.get_nodes())
  poblacion = generar_poblacion(Nodos,N)
  
  #Inicializamos valores para la mejor solucion
  (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)

  
  #Condicion de parada
  parar = False
  n=0
  #Inciamos el cliclo de generaciones
  while(parar == False) :
    
    #Cruce de la poblacion(incluye mutación)
    poblacion = Cruzar(poblacion, mutacion, problem)
  
    #Seleccionamos la población
    poblacion = Seleccionar(problem,poblacion, N, elitismo)
      
    #Evaluamos la nueva población
    (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)
  
    print("Generacion #", n, "\nLa mejor solución es:" , mejor_solucion, "\ncon distancia " , mejor_distancia, "\n")
 
    #Numero de generaciones. Criterio de parada
    if n==generaciones:
      parar = True
    n +=1
    
  return mejor_solucion
  
#1º intento  :2113
#sol = algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=200)

#2º intento. Aumentamos la poblacion:1654
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.1,generaciones=200)

#3º intento. Aumentamos las generaciones:2055
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.1,generaciones=250)

#4º intento. Aumentamos el elitismo: 1728
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.40,generaciones=250)

#5º intento. Aumentamos la mutacion : 1629 [0, 29, 9, 21, 40, 24, 39, 22, 38, 30, 28, 4, 18, 26, 5, 6, 32, 34, 33, 20, 35, 36, 31, 17, 37, 7, 8, 23, 41, 10, 25, 11, 12, 13, 19, 14, 16, 15, 1, 3, 2, 27] 
sol = algoritmo_genetico(problem=problem,N=500,mutacion=.3,elitismo=.40,generaciones=250)

Generacion # 0 
La mejor solución es: [0, 33, 30, 34, 20, 23, 14, 13, 40, 39, 24, 32, 19, 2, 27, 22, 10, 18, 9, 16, 15, 29, 3, 8, 21, 38, 36, 35, 5, 26, 28, 1, 6, 7, 37, 17, 41, 11, 4, 12, 25, 31] 
con distancia  3800 

Generacion # 1 
La mejor solución es: [0, 22, 21, 23, 12, 11, 19, 13, 5, 1, 35, 18, 2, 25, 26, 6, 24, 20, 34, 33, 10, 41, 14, 29, 27, 4, 15, 30, 3, 28, 17, 32, 36, 16, 37, 7, 8, 9, 40, 38, 39, 31] 
con distancia  3674 

Generacion # 2 
La mejor solución es: [0, 22, 21, 23, 12, 11, 19, 13, 5, 1, 35, 18, 2, 25, 26, 6, 24, 20, 34, 33, 10, 41, 14, 29, 27, 4, 15, 30, 3, 28, 17, 32, 36, 16, 37, 7, 8, 9, 40, 38, 39, 31] 
con distancia  3674 

Generacion # 3 
La mejor solución es: [0, 29, 36, 27, 1, 18, 26, 2, 24, 39, 9, 22, 33, 20, 35, 13, 37, 32, 4, 6, 7, 38, 34, 30, 3, 12, 10, 8, 5, 17, 21, 40, 23, 41, 11, 14, 16, 19, 15, 25, 31, 28] 
con distancia  3475 

Generacion # 4 
La mejor solución es: [0, 29, 35, 27, 1, 18, 26, 2, 24, 39, 9, 22, 33, 20, 36, 13, 37, 32, 4, 6, 7, 38, 